In [2]:
# for MOSART grids, find the one corresponding to outlet
# note mosart river network can be wrong so the area match may not be good
import pandas as pd
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import xarray as xr

In [4]:
# load basin info
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
df_info = pd.read_csv(file)
df_info.head()

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,Source,...,RawTDXarea,HSTDXarea,MHTDXarea,RawCAMELSbias,RawTDXbias,HSTDXbias,MHTDXbias,final_bias,geometry,file_obsQ
0,1013500,0.0,-68.56551,47.01169,2.303988e+09,277.493490,NaN,NaN,647993,RawTDX,...,2246.946668,2246.628415,2246.132802,0.022767,-0.002554,-0.002695,-0.002915,0.002554,POLYGON ((-68.84000529335947 46.73746031546882...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
1,1022500,0.0,-68.07313,44.79691,6.203873e+08,103.604200,NaN,NaN,312624,HSTDX,...,615.173865,586.427404,586.602450,0.081568,0.072479,0.022363,0.022668,0.022363,POLYGON ((-68.13005554673525 44.64716666195261...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
2,1030500,0.0,-68.14985,45.80527,3.676155e+09,174.433896,NaN,NaN,662248,RawCAMELS,...,3654.475429,3653.788946,3653.845830,-0.000004,-0.005901,-0.006088,-0.006073,0.000004,POLYGON ((-68.35800170861474 45.67902755716805...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
3,1031500,0.0,-69.58119,45.23568,7.665447e+08,304.743490,NaN,NaN,309614,RawCAMELS,...,775.632787,772.001421,772.168545,-0.003258,0.008560,0.003838,0.004055,0.003258,POLYGON ((-69.67646789509475 45.14558029212971...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...
4,1047000,0.0,-70.16213,44.98744,9.049562e+08,379.779978,NaN,NaN,310157,RawCAMELS,...,914.491471,914.237495,913.814091,-0.004558,0.005931,0.005651,0.005185,0.004558,POLYGON ((-70.10847473135688 45.21669387774366...,/glade/work/guoqiang/CTSM_cases/CAMELS_Calib/C...


In [39]:
matchinfo = []

for b in range(len(df_info)):

    # basin info
    latb = df_info.iloc[b]['latoutlet']
    lonb = df_info.iloc[b]['lonoutlet']
    areab = df_info.iloc[b]['areaUSGS'] # km2

    # mosart info
    file_mo = f'/glade/work/guoqiang/CTSM_CAMELS/data_mosart/CAMELS_level1_{b}_MOSART_routing_0.125nldas2.nc'
    with xr.open_dataset(file_mo) as ds_mo:
        lon = ds_mo.longxy.values[0,:]
        lat = ds_mo.latixy.values[:,0]
        areaup = ds_mo.areaTotal.values
    
    # find outlet grid
    latdiff = np.min( np.abs(lat - latb) )
    latind = np.argmin( np.abs(lat - latb) )
    latgrid = lat[latind]

    londiff = np.min( np.abs(lon - lonb) )
    lonind = np.argmin( np.abs(lon - lonb) )
    longrid = lon[lonind]
    
    areaup_mo = areaup[latind, lonind] / 1e6

    # print(areab, areaup_mo)
    # print(latdiff, londiff)

    matchinfo.append([latb, lonb, areab, latdiff, londiff, latind, lonind, areaup_mo])

In [52]:
matchinfo = np.array(matchinfo)
df_matchinfo = pd.DataFrame(matchinfo, columns=['latoutlet','lonoutlet','areaUSGS','latdiff','londiff','latind','lonind','areaup'])
df_matchinfo.to_csv('CAMELS_level1_MOSART_matchinfo.csv', index=False)

In [40]:
# # plot one basin

# basin = 0
# file_mesh = f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/disaggregation/corrected_HCDN_nhru_final_671_buff_fix_holes.CAMELSandTDX_areabias_fix.simp0.001.level1_polygons_neighbor_group_esmf_mesh_{basin}.nc'
# ds_basin = xr.load_dataset(file_mesh)
# nodeCoords = ds_basin.nodeCoords.values

# dsmo2 = ds_mo.copy()
# dsmo2['lon'] = lon
# dsmo2.areaTotal.plot()
# plt.scatter(nodeCoords[:,0]-360, nodeCoords[:,1], 20, color='w')
# plt.scatter(lonb, latb, 20, color='r')